## Modules

In [ ]:
import sys; sys.path.append("../modules")
import random
from itertools import product

import numpy as np
import sympy; sympy.init_printing()

import Permutation as pm
#from Grid import *

from tqdm.notebook import tqdm
# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()

from knot_floer_homology import *
#help(pd_to_hfk)

from GridPythonModule import *

In [ ]:
count = 0
for _ in tqdm(range(1000)):
    G = generate_random_grid(20)
    if number_of_components(G) == 1:
        # draw_grid(G)
        count += 1
print(count)

## Classes

In [ ]:
class Edge:
    '''Edge class expressing edges in a grid diagram.'''
    def __init__(self, hv, index, xoends):
        self.hv = hv
        self.index = index
        self.xoends = xoends
        self.xend, self.oend = xoends
        self.direction = 1 if not (self.xend < self.oend)^(self.hv == 'h') else -1

class Crossing:
    def __init__(self, hedge, vedge):
        self.hedge, self.vedge = hedge, vedge
        self.sign = self.get_sign()
        self.index = (hedge.index, vedge.index)
        
    def get_sign(self):
        signs = [-1]
        he, ve = self.hedge, self.vedge
        for e, i in [(he, ve.index), (ve, he.index)]:
            e.sorted_end = e.xoends if e.xend < e.oend else e.xoends[::-1]
            sign = e.direction if e.sorted_end[0] < i < e.sorted_end[1] else 0
            signs.append(sign)
        return np.prod(signs)

    def __eq__(self, other_cr):
        return other_cr == self.index

class Grid:
    def __init__(self, xlist, olist):
        self.orig = [xlist, olist]
        self.size = len(xlist)
        self.xprm, self.oprm = pm.Permutation(xlist), pm.Permutation(olist)
        self.hedges = [Edge('h', i, (self.xprm.act(i), self.oprm.act(i))) for i in range(self.size)]
        self.vedges = [Edge('v', i, (self.xprm.inverse().act(i), self.oprm.inverse().act(i))) for i in range(self.size)]
        self.crossings = self.get_crossings()
        
    def get_crossings(self):
        crossings = []
        for he, ve in product(self.hedges, self.vedges):
            if not (he.index in [0, self.size-1] or ve.index in [0, self.size-1]):
                cr = Crossing(he, ve)
                crossings.append(cr) if not cr.sign == 0 else None
        return crossings

    def draw(self):
        draw_grid(self.orig, markings='XO')

    def num_of_comps(self):
        number_of_components(self.orig)

## Scratch

In [ ]:
#mgrid = Grid(*[[3, 2, 5, 4, 7, 6, 0, 1], [0, 4, 3, 6, 5, 1, 2, 7]]); 
flag = True
while flag:
    G = generate_random_grid(12)
    if number_of_components(G) == 1:
        flag = False
mgrid = Grid(*G)
mgrid.draw()

#for cr in mgrid.crossings:
#    print([(e.index, e.xoends) for e in [cr.hedge, cr.vedge]], cr.sign)

edge_trace = [mgrid.hedges[0]]
closed = False
while not closed:
    edge_trace.append(mgrid.vedges[edge_trace[-1].oend])
    edge_trace.append(mgrid.hedges[edge_trace[-1].xend])
    if edge_trace[-1] == edge_trace[0]:
        closed = True
del edge_trace[-1]

#print([f"{e.hv}{e.index}{e.xoends}" for e in edge_trace])

In [ ]:
dummy_cr = Crossing(mgrid.hedges[0], mgrid.vedges[0])
edge_trace_divided = [[(edge_trace[-1],dummy_cr)]]

count = 0
for e in edge_trace:
    edge_trace_divided[-1].append((e, dummy_cr))
    edge_divided = []
    for cr in mgrid.crossings:
        if e in [cr.hedge, cr.vedge]:
            edge_divided.append((e, cr))
    for ed in edge_divided[::e.direction]:
        edge_trace_divided.append([ed])

#for ll in edge_trace_divided:
#    print([f"{e.hv}{e.index}{cr.index}" for e, cr in ll])

PD = []
modulo = len(edge_trace_divided)-1
for cr in mgrid.crossings:
    connected = {'w': None, 's': None, 'e': None, 'n': None}
    for i, l in enumerate(edge_trace_divided):
        e, x = l[0]
        if x == cr:
            pair = (i%modulo, (i-1)%modulo)
            if e.hv == 'h':
                connected['e'], connected['w'] = pair[::e.direction]
                flag = e.direction
            else:
                connected['n'], connected['s'] = pair[::e.direction]
    code = list(connected.values())
    if flag == -1:
        code = code[2:]+code[:2]
    PD.append(code)        
print(PD)

In [ ]:
def reduce_once(PD):
    for i, l in enumerate(PD):
        if len(set(l)) < len(l):
            pair = [v for v in l if l.count(v) == 1]
            del PD[i]
            PD = [list(map(lambda x: min(pair) if x==max(pair) else x, l)) for l in PD]
            break
    return PD

def reduce(PD):
    prev = len(PD)
    flag = True
    while flag and prev > 0:
        PD = reduce_once(PD)
        if len(PD) == prev:
            flag = False
        else:
            prev = len(PD)
    flatten = list(set(sum(PD, [])))
    return [list(map(lambda x: flatten.index(x), l)) for l in PD]
    
result = reduce(PD)
print(result)

In [ ]:
print(pd_to_hfk(list(sum(PD, []))))